In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('Creditcard_data.csv')

In [5]:
# Balance the Dataset
majority = data[data['Class'] == 0]
minority = data[data['Class'] == 1]
minority_oversampled = minority.sample(len(majority), replace=True, random_state=42)
balanced_data = pd.concat([majority, minority_oversampled])

In [6]:
# Create Samples
sample_sizes = [int(len(balanced_data) * 0.2)] * 5
samples = [balanced_data.sample(n=sample_sizes[i], random_state=i) for i in range(5)]

In [7]:
from sklearn.model_selection import train_test_split

# Apply Sampling Techniques
simple_random_sample = samples[0]
stratified_sample = train_test_split(balanced_data, stratify=balanced_data['Class'], test_size=0.2, random_state=42)[0]
clusters = balanced_data.groupby('V1')
cluster_sample = clusters.apply(lambda x: x.sample(1)).reset_index(drop=True)
k = 10
systematic_sample = balanced_data.iloc[::k, :]
weighted_sample = balanced_data.sample(n=sample_sizes[0], weights='Amount', random_state=42)

C:\Users\win 10\AppData\Local\Temp\ipykernel_13384\4004682169.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cluster_sample = clusters.apply(lambda x: x.sample(1)).reset_index(drop=True)


In [8]:
sampling_methods = {
    "Simple Random": simple_random_sample,
    "Stratified": pd.DataFrame(stratified_sample),
    "Cluster": cluster_sample,
    "Systematic": systematic_sample,
    "Weighted": weighted_sample
}

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

models = {
    "M1": LogisticRegression(),
    "M2": DecisionTreeClassifier(),
    "M3": SVC(),
    "M4": RandomForestClassifier(),
    "M5": KNeighborsClassifier()
}

In [18]:
# Train and Evaluate Models
accuracy_matrix = pd.DataFrame(columns=sampling_methods.keys(), index=models.keys())

best_accuracy = 0
best_combinations = []

In [19]:
from sklearn.metrics import accuracy_score

def train_evaluate_model(model, data_sample):
    X = data_sample.drop('Class', axis=1)
    y = data_sample['Class']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [20]:
# Evaluate each model on each sample
for model_name, model in models.items():
    for sample_name, sample_data in sampling_methods.items():
        accuracy = train_evaluate_model(model, sample_data)
        accuracy_matrix.loc[model_name, sample_name] = accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_combinations = [(model_name, sample_name)]
        elif accuracy == best_accuracy:
            best_combinations.append((model_name, sample_name))

c:\Users\win 10\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\win 10\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

In [21]:
# Print Accuracy Matrix
print("\nAccuracy Matrix:")
print(accuracy_matrix)


Accuracy Matrix:
   Simple Random Stratified   Cluster Systematic  Weighted
M1      0.956522   0.923497   0.96087   0.782609  0.978261
M2      0.978261   0.986339   0.96087   0.956522   0.98913
M3      0.630435   0.693989  0.973913   0.586957  0.891304
M4           1.0        1.0  0.973913        1.0   0.98913
M5      0.923913   0.972678  0.973913   0.782609  0.956522


In [22]:
# Print Best Models and Samples
print(f"\nBest Accuracy: {best_accuracy:.2f} achieved by:")
for model, sample in best_combinations:
    print(f"Model '{model}' on Sample '{sample}'")


Best Accuracy: 1.00 achieved by:
Model 'M4' on Sample 'Simple Random'
Model 'M4' on Sample 'Stratified'
Model 'M4' on Sample 'Systematic'
